In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ase
import copy
import os
import pickle
import sys
import numpy as np
import json

np.set_printoptions(precision=3, suppress=True)
# uncomment and replace with correct path if there are import errors
# sys.path.append("/path/to/surface-sampling/")
# sys.path.append("/path/to/NeuralForceField/")
# os.environ["LAMMPS_POTENTIALS"] = "/path/to/lammps/potentials/"
# os.environ["LAMMPS_COMMAND"] ="/path/to/lammps/src/lmp_serial"
# os.environ["ASE_LAMMPSRUN_COMMAND"] = os.environ["LAMMPS_COMMAND"]

from mcmc import MCMC
from mcmc.system import SurfaceSystem

from time import perf_counter

Initialize test slab and parameters

In [3]:
# Load prepared pristine slab
element = []
slab_pkl = open("../tutorials/data/SrTiO3_001_2x2_pristine_slab.pkl", "rb")

slab = pickle.load(slab_pkl)

ref_slab_files = [
    "/mnt/data0/dux/2023_paper_figures/data/final_data/SrTiO3/selected_terms/optim_slab_run_001_O44Sr12Ti16.cif",
    "/mnt/data0/dux/2023_paper_figures/data/final_data/SrTiO3/selected_terms/optim_slab_run_001_O36Sr12Ti12.cif",
    "/mnt/data0/dux/2023_paper_figures/data/final_data/SrTiO3/selected_terms/optim_slab_run_001_O40Sr16Ti12.cif"
]

ref_slabs = [ase.io.read(f) for f in ref_slab_files]

In [11]:
system_settings = {
    "surface_name": 'SrTiO3(001)',
    "relax_atoms": False,
    "relax_steps": 5,
    "offset": True,
    "offset_data": os.path.join(
        "../tutorials",
        "data/nff",
        "offset_data.json",
    ),
    "optimizer": "BFGS",
    "cutoff": 5.0,
    "calc_name": "kim",
    "near_reduce": 0.01,
    "planar_distance": 1.55,
    "no_obtuse_hollow": True,
}

sampling_settings = {
    "alpha": 1.0, # no annealing
    "temperature": 1.0, # in terms of kbT
    "chem_pots": {"Sr": -2, "Ti": 0, "O": 0}, # change to 0 here
    "num_sweeps": 10,
    "sweep_size": 64,
}

calc_params = {
    "chem_pots": sampling_settings["chem_pots"],
    "offset_data": json.load(open(system_settings["offset_data"], "r")),
}

Obtain adsorption sites

In [12]:
from pymatgen.analysis.adsorption import AdsorbateSiteFinder
from pymatgen.io.ase import AseAtomsAdaptor

pristine_slab = slab.copy()
pristine_pmg_slab = AseAtomsAdaptor.get_structure(pristine_slab)
site_finder = AdsorbateSiteFinder(pristine_pmg_slab)
# new standardized params
# ads_positions = site_finder.find_adsorption_sites(
#     put_inside=True,
#     symm_reduce=False,
#     near_reduce=system_settings['near_reduce'],
#     distance=system_settings["planar_distance"],
#     no_obtuse_hollow=system_settings['no_obtuse_hollow'],
# )["all"]

# old params
sites = site_finder.find_adsorption_sites(
    distance=1.55, put_inside=False, symm_reduce=False, no_obtuse_hollow=False
)
ads_positions = sites["all"]

print("adsorption coordinates are")
print(ads_positions)

sweep_size = len(ads_positions)

adsorption coordinates are
[array([-0.   , -0.029, 18.87 ]), array([ 1.968,  1.951, 18.829]), array([ 1.968,  0.146, 18.782]), array([-0.   ,  3.956, 18.87 ]), array([ 1.968,  5.936, 18.829]), array([ 1.968,  4.131, 18.782]), array([ 3.936, -0.029, 18.87 ]), array([ 5.903,  1.951, 18.829]), array([ 5.903,  0.146, 18.782]), array([ 3.936,  3.956, 18.87 ]), array([ 5.903,  5.936, 18.829]), array([ 5.903,  4.131, 18.782]), array([ 9.839, 30.936, 18.806]), array([ 8.855, 30.848, 18.85 ]), array([29.517, 30.936, 18.806]), array([30.5  , 30.848, 18.85 ]), array([14.758, 15.998, 18.826]), array([15.742, 13.906, 18.829]), array([30.5  , 28.856, 18.85 ]), array([ 8.855, 28.856, 18.85 ]), array([10.823, 30.848, 18.85 ]), array([14.758, 24.871, 18.85 ]), array([16.726, 24.871, 18.85 ]), array([15.742, 25.861, 18.829]), array([30.5  , 27.953, 18.826]), array([29.517, 28.943, 18.806]), array([28.533, 28.856, 18.85 ]), array([28.533, 27.953, 18.826]), array([12.79 , 15.998, 18.826]), array([12.79 , 

Set up NFF (calculator). We are using neural network weights from our Zenodo dataset (https://zenodo.org/record/7927039). The ensemble requires an `offset_data.json` file

In [13]:
import torch
from nff.io import NeuralFF, AtomsBatch, EnsembleNFF
from mcmc.calculators import EnsembleNFFSurface

if torch.cuda.is_available():
    DEVICE = 0
else:
    DEVICE = "cpu"

# requires an ensemble of models in this path and an `offset_data.json` file
nnids = ["model01", "model02", "model03"]
model_dirs = [
    os.path.join(
        "../tutorials",
        "data/nff",
        str(x),
        "best_model",
    )
    for x in nnids
]

models = []
for modeldir in model_dirs:
    m = NeuralFF.from_file(modeldir, device=DEVICE).model
    models.append(m)

nff_surf_calc = EnsembleNFFSurface(models, device=DEVICE)
nff_surf_calc.set(**calc_params)

offset data: {'bulk_energies': {'O': -0.17747231201, 'Sr': -0.06043637668, 'SrTiO3': -1.470008697358702}, 'stoidict': {'Sr': 0.49995161381315867, 'Ti': -0.0637500349111578, 'O': -0.31241304903276834, 'offset': -11.324476454433157}, 'stoics': {'Sr': 1, 'Ti': 1, 'O': 3}, 'ref_formula': 'SrTiO3', 'ref_element': 'Ti'} is set from parameters
chemical potentials: {'Sr': -2, 'Ti': 0, 'O': 0} are set from parameters
offset data: {'bulk_energies': {'O': -0.17747231201, 'Sr': -0.06043637668, 'SrTiO3': -1.470008697358702}, 'stoidict': {'Sr': 0.49995161381315867, 'Ti': -0.0637500349111578, 'O': -0.31241304903276834, 'offset': -11.324476454433157}, 'stoics': {'Sr': 1, 'Ti': 1, 'O': 3}, 'ref_formula': 'SrTiO3', 'ref_element': 'Ti'} is set from parameters


Initialize surface system

In [14]:
# set attributes
slab_batch = AtomsBatch(
    positions=slab.positions,
    numbers=slab.numbers,
    cell=slab.cell,
    pbc=True,
    cutoff=system_settings["cutoff"],
    props={"energy": 0, "energy_grad": []},
    calculator=nff_surf_calc,
    requires_large_offsets=True,
    directed=True,
    device=DEVICE,
)

# fix bulk atoms
num_bulk_atoms = len(slab_batch)
bulk_indices = list(range(num_bulk_atoms))
print(f"bulk indices {bulk_indices}")
surf_indices = slab.get_surface_atoms()

fix_indices = list(set(bulk_indices) - set(surf_indices))
print(f"fix indices {fix_indices}")

# bulk atoms are 0 and surface atoms are 1
tags = np.ones(num_bulk_atoms)
tags[fix_indices] = 0
slab_batch.set_tags(tags)
print(f"tags {tags}")

surface = SurfaceSystem(slab_batch, ads_positions, nff_surf_calc, system_info=system_settings)
surface.all_atoms.write("SrTiO3_001_2x2_all_virtual_ads.cif")

2024-02-08 21:30:50,664|INFO|initial state is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2024-02-08 21:30:50,665|INFO|number of pristine atoms is 60
2024-02-08 21:30:50,666|INFO|bulk indices are [ 0  1  2  3  4  5  6  9 10 11 12 13 15 16 17 18 19 20 21 24 25 26 27 28
 30 31 32 33 34 35 36 39 40 41 42 43 45 46 47 48 49 50 51 54 55 56 57 58]
2024-02-08 21:30:50,668|INFO|surface indices are [ 7  8 14 22 23 29 37 38 44 52 53 59]


bulk indices [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
fix indices [0, 1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 54, 55, 56, 57, 58]
tags [0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1.]
ads coords is [-0.    -0.029 18.87 ]
ads coords is [ 1.968  1.951 18.829]
ads coords is [ 1.968  0.146 18.782]
ads coords is [-0.     3.956 18.87 ]
ads coords is [ 1.968  5.936 18.829]
ads coords is [ 1.968  4.131 18.782]
ads coords is [ 3.936 -0.029 18.87 ]
ads coords is [ 5.903  1.951 18.829]
ads coords is [ 5.903  0.146 18.782]
ads coords is [ 3.936  3.956 18.87 ]


In [15]:
from mcmc.energy import slab_energy
slab_energy(surface, offset=True, offset_data=system_settings["offset_data"])

(12.49005126953125, 0.30546674132347107, 0.0, 0.0, [])

Test out other reference slabs

In [16]:
ref_slab_batches = [AtomsBatch(
    positions=slab.positions,
    numbers=slab.numbers,
    cell=slab.cell,
    pbc=True,
    cutoff=system_settings["cutoff"],
    props={"energy": 0, "energy_grad": []},
    calculator=nff_surf_calc,
    requires_large_offsets=True,
    directed=True,
    device=DEVICE,
) for slab in ref_slabs]


ref_surfs = []
# additional atoms are adsorbates and should be labelled as 2
for ref_slab_batch in ref_slab_batches:
    num_ads = len(ref_slab_batch) - len(tags)
    curr_tags = np.int32(tags.tolist() + [2] * num_ads)
    print(f"tags {curr_tags}")
    ref_slab_batch.set_tags(curr_tags)
    ref_surf = SurfaceSystem(ref_slab_batch, ads_positions, nff_surf_calc, system_info=system_settings)
    ref_surfs.append(ref_surf)


2024-02-08 21:30:51,384|INFO|initial state is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2024-02-08 21:30:51,385|INFO|number of pristine atoms is 72
2024-02-08 21:30:51,386|INFO|bulk indices are [ 0  1  2  3  4  5  6  9 10 11 12 13 15 16 17 18 19 20 21 24 25 26 27 28
 30 31 32 33 34 35 36 39 40 41 42 43 45 46 47 48 49 50 51 54 55 56 57 58]
2024-02-08 21:30:51,387|INFO|surface indices are [ 7  8 14 22 23 29 37 38 44 52 53 59]


tags [0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 2 2 2 2 2 2 2 2 2 2 2 2]
ads coords is [-0.    -0.029 18.87 ]
ads coords is [ 1.968  1.951 18.829]
ads coords is [ 1.968  0.146 18.782]
ads coords is [-0.     3.956 18.87 ]
ads coords is [ 1.968  5.936 18.829]
ads coords is [ 1.968  4.131 18.782]
ads coords is [ 3.936 -0.029 18.87 ]
ads coords is [ 5.903  1.951 18.829]
ads coords is [ 5.903  0.146 18.782]
ads coords is [ 3.936  3.956 18.87 ]
ads coords is [ 5.903  5.936 18.829]
ads coords is [ 5.903  4.131 18.782]
ads coords is [ 9.839 30.936 18.806]
ads coords is [ 8.855 30.848 18.85 ]
ads coords is [29.517 30.936 18.806]
ads coords is [30.5   30.848 18.85 ]
ads coords is [14.758 15.998 18.826]
ads coords is [15.742 13.906 18.829]
ads coords is [30.5   28.856 18.85 ]
ads coords is [ 8.855 28.856 18.85 ]
ads coords is [10.823 30.848 18.85 ]
ads coords is [14.758 24.871 18.85 ]
ads coords is [16.726 24.871 18.85

2024-02-08 21:30:51,627|INFO|initial state is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2024-02-08 21:30:51,628|INFO|number of pristine atoms is 60
2024-02-08 21:30:51,629|INFO|bulk indices are [ 0  1  2  3  4  5  6  9 10 11 12 13 15 16 17 18 19 20 21 24 25 26 27 28
 30 31 32 33 34 35 36 39 40 41 42 43 45 46 47 48 49 50 51 54 55 56 57 58]
2024-02-08 21:30:51,629|INFO|surface indices are [ 7  8 14 22 23 29 37 38 44 52 53 59]
2024-02-08 21:30:51,712|INFO|initial state is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2024-02-08 21:30:51,713|INFO|number of pristine atoms is 68
2024-02-08 21:30:51,714|INFO|bulk indices are [ 0  1  2  3  4  5  6  9 10 11 12 13 15 16 17 18 19 20 21 24 25 26 27 28
 30 31 32 33 34 35 36 39 40 41 42 43 45 46 47 48 49 50 51 54 55 56 57 58]
2024-02-08 21:30:51,7

ads coords is [-0.    -0.029 18.87 ]
ads coords is [ 1.968  1.951 18.829]
ads coords is [ 1.968  0.146 18.782]
ads coords is [-0.     3.956 18.87 ]
ads coords is [ 1.968  5.936 18.829]
ads coords is [ 1.968  4.131 18.782]
ads coords is [ 3.936 -0.029 18.87 ]
ads coords is [ 5.903  1.951 18.829]
ads coords is [ 5.903  0.146 18.782]
ads coords is [ 3.936  3.956 18.87 ]
ads coords is [ 5.903  5.936 18.829]
ads coords is [ 5.903  4.131 18.782]
ads coords is [ 9.839 30.936 18.806]
ads coords is [ 8.855 30.848 18.85 ]
ads coords is [29.517 30.936 18.806]
ads coords is [30.5   30.848 18.85 ]
ads coords is [14.758 15.998 18.826]
ads coords is [15.742 13.906 18.829]
ads coords is [30.5   28.856 18.85 ]
ads coords is [ 8.855 28.856 18.85 ]
ads coords is [10.823 30.848 18.85 ]
ads coords is [14.758 24.871 18.85 ]
ads coords is [16.726 24.871 18.85 ]
ads coords is [15.742 25.861 18.829]
ads coords is [30.5   27.953 18.826]
ads coords is [29.517 28.943 18.806]
ads coords is [28.533 28.856 18.85 ]
a

In [17]:
for surf in ref_surfs:
    results = slab_energy(surf, offset=True, offset_data=system_settings["offset_data"])
    surf_energy = results[0]
    print(f"energy of reference slab is {surf_energy}")

energy of reference slab is 35.99249267578125
energy of reference slab is 12.486358642578125
energy of reference slab is -4.78692626953125
